In [ ]:
!git clone https://github.com/0601p/Model-Compression.git

In [ ]:
!pip install torch-pruning # v1.0.0

In [ ]:
import torch
import torch.nn.utils.prune as prune
import torchvision
import random
import sys
import torch_pruning
sys.path.append("/content/Model-Compression")

from model import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [ ]:
learning_rate = 1e-4
batch_size = 512
epoch = 20

In [ ]:
root = "./CIFAR100"
transform = torchvision.transforms.ToTensor()
data_train = torchvision.datasets.CIFAR100(root, train = True, transform = transform, download = True)
data_test = torchvision.datasets.CIFAR100(root, train = False, transform = transform, download = True)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=100).to(device)
checkpoint = torch.load("/content/Model-Compression/model.pth")
model.load_state_dict(checkpoint)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

Code for Training and Eval

In [ ]:
def train_one_epoch(print_result = False):
    model.train()
    loss_sum = 0.0
    accuracy_sum = 0.0
    length = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()

        pred = model(X)
        loss = criterion(pred, Y)
        pred_idx = torch.argmax(pred, 1)
        loss_sum += loss.item()
        accuracy_sum += torch.sum((pred_idx == Y).float()).item()
        length += X.size(0)

        loss.backward()
        optimizer.step()

    if(print_result):
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)

In [ ]:
def eval():
    with torch.no_grad():
        model.eval()
        loss_sum = 0.0
        accuracy_sum = 0.0
        length = 0

        for X, Y in train_loader:
            X = X.to(device)
            Y = Y.to(device)

            pred = model(X)
            loss = criterion(pred, Y)
            pred_idx = torch.argmax(pred, 1)
            loss_sum += loss.item()
            accuracy_sum += torch.sum((pred_idx == Y).float()).item()
            length += X.size(0)
            
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)

In [ ]:
def train():
    for i in range(epoch):
        print("EPOCH[" + str(i + 1) + "]")
        print("==== train ====")
        train_one_epoch(print_result=True)
        
        print("==== eval ====")
        eval()

Code for Pruning

In [ ]:
""" code for pruning specific filters

DG = torch_pruning.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1, 3, 32, 32).to("cuda"))
pruning_idxs = [0, 1, 2, 3, 4] # remove filter 0 ~ 4
pruning_group = DG.get_pruning_group(model.conv1, torch_pruning.prune_conv_out_channels, idxs = pruning_idxs)
print(pruning_group)
if DG.check_pruning_group(pruning_group):
    pruning_group.exec()
"""

In [ ]:
# code for pruning whole model

example_inputs = torch.randn(1, 3, 32, 32).to("cuda")
imp = torch_pruning.importance.MagnitudeImportance(p = 2)
ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 100:
        ignored_layers.append(m)

#ch_sparsity = 0.5 means 50% of the filters are pruned
pruner = torch_pruning.pruner.MagnitudePruner(model=model, example_inputs=example_inputs, importance=imp, iterative_steps=5, ch_sparsity=0.5, ignored_layers=ignored_layers)

base_macs, base_nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)
print("basemacs :", base_macs, "basenparams :", base_nparams)
for i in range(5):
    pruner.step()
    macs, nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)
    print("iter [", i, "] macs :", macs, "nparams :", nparams)

    #finetune
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    for j in range(10):
        train_one_epoch(print_result = True)
    
    eval()


In [ ]:
eval()